In [1]:
import keyboard
import os
import tempfile
import re
import time
import numpy as np
import openai
import sounddevice as sd
import soundfile as sf

from elevenlabs import generate, play, set_api_key
from IPython.display import Audio, display
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel

In [2]:
api_key = '<>'
openai.api_key = api_key
set_api_key("<>")

In [3]:
fs = 48000  # sample rate
channels = 1  # number of channels

In [4]:
def record_audio(fs,cannels):
    print("Press and hold spacebar to start recording. Release to stop.")
    while True:
        if keyboard.is_pressed('space'):
            print("Recording...")
            recorded_chunks = []
            with sd.InputStream(samplerate=fs, channels=channels) as stream:
                while keyboard.is_pressed('space'):
                    audio_chunk, overflowed = stream.read(fs)
                    recorded_chunks.append(audio_chunk)
            recording = np.concatenate(recorded_chunks, axis=0)
            print("Finished recording.")
            return recording
        sd.sleep(100)  # Sleep for a short duration before checking again

def transcribe_audio(recording, fs):
    # This function is kept for completeness. You might not need this if you're not transcribing.
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_audio:
        sf.write(temp_audio.name, recording, fs)
        # Transcription part should go here if needed
        temp_audio.close()


In [5]:
def transcribe_audio(recording, fs):
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_audio:
        sf.write(temp_audio.name, recording, fs)
        temp_audio.close()
        with open(temp_audio.name, "rb") as audio_file:
            transcript = openai.Audio.transcribe(model="whisper-1",
                                                 file=audio_file,
                                                 language="en")
        os.remove(temp_audio.name)
    return transcript["text"].strip()

In [6]:
def play_generated_audio_doctor(text, voice="Callum", model="eleven_monolingual_v1"):
    audio = generate(text=text, voice=voice, model=model)
    display(Audio(data=audio, autoplay=True))

In [7]:
def play_generated_audio_nurse(text, voice="Charlotte", model="eleven_monolingual_v1"):
    audio = generate(text=text, voice=voice, model=model)
    display(Audio(data=audio, autoplay=True))

In [8]:
model = ChatOpenAI(temperature=0,
                   model_name="gpt-4",
                   max_tokens=200,
                   openai_api_key=api_key,
                  )

In [9]:
memory1 = ConversationBufferMemory(memory_key="chat_history")

In [10]:
memory2 = ConversationBufferMemory(memory_key="chat_history")

In [11]:
template1 = """
As a virtual nurse, your objective is to gather the following patient details:
- Name
- Reason for Call
- Severity of Symptoms
- Current Medications
- Known Allergies
- Recent Medical Events

Engage the patient in a relaxed conversation. Pose your questions one by one, ensuring you await a response after each question. Feel free to indulge in casual chat for a comfortable environment. 

Once you've collected all the information, wrap up the chat gracefully and conect patient directly to doctor.

{chat_history}
Patient: {human_input}
Virtual Nurse: 
"""

prompt1 = PromptTemplate(
    input_variables=["chat_history", "human_input"],
    template=template1
)

In [12]:
template2 = """
You are a Virtual Health Assistant. Your role is to provide health information, guidance, and active listening. Engage the patient in a natural, conversational manner. Ask follow-up questions one at a time to delve deeper into the patient's condition. Once you've gathered sufficient information, provide your final assessment and diagnosis. Remember to mention that this is solely for informational purposes and does not replace a professional medical diagnosis. Recommend over-the-counter solutions if appropriate, but if the patient's condition seems to require professional medical attention, guide them to consult with a real doctor. Start by introducing yourself and proceed with a gentle, inquiry-based approach to understand the patient's situation better.
{chat_history}
Patient: {human_input}
Virtual Health Assistant: 
"""

prompt2 = PromptTemplate(
    input_variables=["chat_history", "human_input"],
    template=template2
)

In [13]:
template3 = """
Create a concise summary from the patient and nurse conversation, including the patient's name, reason for call, severity of symptoms, current medications, known allergies, and recent medical events.
Patient and Nurse conversation: {data}
Output: """

prompt3 = PromptTemplate(
    input_variables=["data"],
    template=template3
)

In [14]:
chain_nurse=llm_chain = LLMChain(llm=model,
                                 prompt=prompt1,
                                 verbose=True,
                                 memory=memory1
                                )

In [ ]:
chain_doctor=llm_chain = LLMChain(llm=model,
                             prompt=prompt2,
                             verbose=True,
                             memory=memory2
                            )



In [ ]:
chain_data=llm_chain = LLMChain(llm=model,
                             prompt=prompt3,
                             verbose=True,
                            )

In [ ]:
def memory_to_string(memory_object):
    # Convert the object to string representation
    string_representation = str(memory_object)
    
    # Extracting the message contents
    human_messages = re.findall(r"HumanMessage\(content='(.*?)'", string_representation)
    ai_messages = re.findall(r"AIMessage\(content=\"(.*?)\"", string_representation)
    
    # Creating a conversation format
    conversation = ""
    for human, ai in zip(human_messages, ai_messages):
        conversation += "Human: " + human + "\n"
        conversation += "AI: " + ai + "\n"
    
    return conversation

In [ ]:

exit_loops = False

response = chain_nurse.predict(human_input="Hi")
play_generated_audio_nurse(response)
while True:
    
    keyboard.wait("space")  # wait for spacebar to be pressed
    recorded_audio = record_audio(fs, channels) 
    message = transcribe_audio(recorded_audio, fs)
    print(f"You: {message}")
    response = chain_nurse.predict(human_input=message)
    play_generated_audio_nurse(response)
    context = memory_to_string(memory1)
    time.sleep(10)
    
    
    
    if "doctor" in context.lower():
        print("Context:", context)
        data = chain_data.predict(data=context)
        print("Model format output :", data)
        response = chain_doctor.predict(human_input=data)
        play_generated_audio_doctor(response)
        while True:
            keyboard.wait("space")  # wait for spacebar to be pressed
            recorded_audio = record_audio(fs, channels) 
            message = transcribe_audio(recorded_audio, fs)

            print(f"You: {message}")


            response = chain_doctor.predict(human_input=message)

            play_generated_audio_doctor(response)
            
            
            if "bye" in context:
                exit_loops = True
                break
    
    if exit_loops == True:
            break